<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2016 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2016 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

# TUTORIAL 4: Working with Legacy HSPF files and functions

This tutorial notebook demonstrates how to use legacy UCI and WDM files to create
an HDF5 file for $\textbf{HSP}^{\textbf{2}}$. It shows how to use legacy PLTGEN files to plot results (like the plots in Tutorial 5.) Finally, it demonstrates how to perform a post run analysis like the HSPF DURANL.

**Tutorial Contents**

 + Section 1: [Importing UCI Files into HDF5](#section1)
 + Section 2: [Importing WDM Files into HDF5](#section2)
 + Section 3: [HSP2 DURANL Functionality](#section3)
 + Section 4: [HSP2 PLTGEN Functionality](#section4)

### Required Python imports  and setup 

In [ ]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

import numpy as np
import pandas as pd
pd.options.display.max_rows    = 18
pd.options.display.max_columns = 20
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

import matplotlib.pyplot as plt
%matplotlib inline

import datetime

import HSP2
import HSP2tools

HSP2tools.reset_tutorial()    # make a new copy of the tutorial's data
HSP2tools.versions()          # display version information below

#### Set the filenames
This first example uses the HSPF test 10 UCI and WDM files.

In [ ]:
uciname = os.path.join('TutorialData', 'test10.uci') 
wdmname = os.path.join('TutorialData', 'test.wdm')

unpackedhdfname = os.path.join('TutorialData', 'unpackedtutorial.h5')
hdfname = os.path.join('TutorialData', 'Tutorial.h5')

**Note**: make sure the tutorial HDF5 files does not exist at the start of this tutorial (since we want to make it here!)

In [ ]:
os.remove(hdfname)
os.listdir('TutorialData')

## Section 1: Importing UCI Files into HDF5<a id='section1'></a>

HSPF was developed for a user input file (the UCI file) based on 80 column punch cards.
The format of data on each card was specific to the type of data it contained.
The sequence files for PERLND, IMPLND, and REACHES contain the format specifications to read the text file lines along
with other information such as default values (for unspecified data), maximum and minimum limits per data element, message
strings defining the meaning/use of each data element, and units type (English or Metric.)
The UCI Reader uses these sequence files to parse the user's UCI file.

The $\textbf{HSP}^\textbf{2}$  UCI Reader function currently is mostly complete except for a few tables that span multiple "cards" or have a special context.  The readUCI module will be fixed to read these tables when the associated HSPF modules are converted to $\textbf{HSP}^\textbf{2}$. 

The UCI Reader will create the HDF5 file if necessary.
If the HDF5 file already exists, it overwrites the UCI corresponding information.

The HDF5 file includes the UCI file information (except obsolete elements) plus some new tables.  For example, there is now a SAVE table for each module in PERLND,
IMPLND, and REACHES that specifies almost every computed timeseries and each segment. A one in the intersection
of a named timeseries and a segment will save the results to the HDF5 file, otherwise it is not saved. This allows fine control for saving results for post run analysis. By default, only the output flux timeseries are saved.
This tutorial shows how to save everything as an example of modifying the SAVE tables from the default.

A few timeseries which can be trivially computed from the other timeseries are not explictly named or saved. The View Perlnd, View Implnd, and View Reaches notebooks provide examples of calculating the "missing" timeseries.

### Run the  UCI Reader
    
The following cell creates the HDF5 file and populates it with the data from the UCI file.

In [ ]:
HSP2tools.makeH5()

In [ ]:
HSP2tools.readUCI(uciname, unpackedhdfname)

Use HDFView or HDFCompass to examine the resulting HDF5 file if you like.

**NOTE:** If the user is only interested in using the Network Tool on a legacy watershed,
then you can stop here. The Network Tool (Tutorial 6) can be run using this HDF5 file.

**NOTE** The readUCI routine can take two options.
+ HSPF - (optional, defaults to False)
   + When HSPF is True, all the data from the UCI file is imported - even if it is not used by $\textbf{HSP}^\textbf{2}$ 
   + When HSPF is False, the obsolete data from the UCI file is ignored.
 
+ metric - (optional, defaults to False).
   + When metric is True, the UCI data is read as being in metric units.
   + When metic is False, the UCI data is read as being in English units.
   

**Section Summary**

 + demonstrated how to create an HDF5 file for HSP$^2$ by reading a legacy UCI file

## Section 2: Importing WDM Files into HDF5<a id='section2'></a>

The WDM reader must be run after the UCI reader because it uses the EXT_SOURCES table from the HDF5 file to determine
which timeseries to extract from the WDM file. It also extracts some metadata from the EXT_SOURCES table which is attached as attributes when the time series is saved to the HDF5 file.

For each timeseries named in the EXT_SOURCES table, the entire timeseries is extracted from the WDM file. It is placed into a Pandas Series and then saved to the HDF5 file.

Each time series WDM name is converted from a number to a string prefaced by "TS". The UCI reader has already adjusted all references to the time series datasets to use this naming convention.

The WDM reader also extracts the metadata from the WDM file and attaches it to the timeseries as attributes.  Pandas also attaches its own metadata.

### Credits, License and Copyright for wdmtoolbox

The UCI Reader uses Tim Cera's **wdmtoolbox**.  This code based on version 0.8.2 was modified by RESPEC because HSPF UCI files that didn't contain a Constituent, Location, or Scenario attrributes will terminate with an error. Since the default HSPF test files don't have these attributes,  HSP2 can't run the test cases without this fix. This was reported to Tim.

The wdmtoolbox is released under a BSD license and Tim Cera retains all rights to his module except as explicitly granted by the license.

The wdmtoolbox is only used by the $\textbf{HSP}^\textbf{2}$  readWDM module.

### Run the  WDM Reader

In [ ]:
HSP2tools.ReadWDM(wdmname, unpackedhdfname)

### Pack the HDF5


You should make the file smaller by repacking it with ptrepack.exe as discussed in Tutorial 3.

In [ ]:
!ptrepack {unpackedhdfname}  {hdfname}

#### View the HDF5 file with HDFView or HDFCompass - Optional

(Close and reopen) HDFView to examine the HDF5 file, tutorial.h5,  which is now complete and ready to run a simulation.  

Using HDFView, click on the TS39 timeseries and look at the
metadata. You can drag the boundary of the metadata panal up to make it bigger or just scroll.  

The WDM file doesn't attach real units to its data. The **EXT_SOURCES** has
a units column (from the UCI data), but it is a string like 'ENGL'.
It is stored in the metadata as 'wdm_units'.  

The 'ENGL' designation is not an accepted units measurement and is also ambiquous.
For example the rainfall timeseries is probably in inches per hour, but internally it is converted to feet per time interval. If the rainfall series comes from a daily source and is disaggregated (via "DIV"), then the units of the original timeseries (inch per day) is different than the converted. 

The real units are marked as '???'. It would be a good practice to annotate the actual units when data is imported. 

Ideally, the new HSP2 could examine the actual units of a timeseries to perform any required aggregation/disaggregation, and mark the resulting timeseries with the new units.  Units would be carried with the data throughout HSP2. But would require the user to alway set the actual units. This was demonstrated in Tutorial 2.

### Run HSPF test10

Now show that this HDF5 file can run the simulation.

In [ ]:
HSP2.run(hdfname)

### Calleg Example

Calleg is a real watershed and has
+ 27 IMPLND segments,
+ 129 PERLND segments,
+ 119 RCHRES segments,
+ 9 years of simulation time with hourly time steps (78,888 timesteps.)

This is watershed is a more realistic indication of HSP$^2$ performance than test10.

In [ ]:
uciname = os.path.join('TutorialData', 'calleg.uci') 
wdmname = os.path.join('TutorialData', 'calleg.wdm')

unpackedhdfname = os.path.join('TutorialData', 'unpackedcalleg.h5')
hdfname = os.path.join('TutorialData', 'calleg.h5.h5')

Repeat the steps above for this watershed

In [ ]:
os.remove(hdfname)

In [ ]:
HSP2tools.readUCI(uciname, unpackedhdfname)
HSP2tools.ReadWDM(wdmname, unpackedhdfname)

!ptrepack {unpackedhdfname}  {hdfname}
!del {unpackedhdfname}

Now run this simulation. This larger simulation will take a few minutes depending on your computer's speed.

In [ ]:
HSP2.run(hdfname)

**Section Summary**

 + demonstrated how to import timeseries from a WDM file into an HDF5 file
 + demonstrated that the new HDF5 file can run the watershed functionality

## Section 3: HSP2 DURANL Functionality<a id='section3'></a>

Pandas makes it easy to answer questions like those in the traditional HSPF DURANL Analysis.
These techniques can be used on any timeseries, external source or computed.

Basically, DURANL answers questions such as

 + Fraction of time spent spent in exceeding a given level for excursions (runs) greater than a specified duration,
 + Time spend between specified levels for excursions greater than a specified duration,
 + Number of excursions exceeding a specified level for excursions greater than a specified duration,
 + Average duration of excursions exceeding a specified level for excursions greater than a specified durations,
 + Standard deviation of duration of excursions exceeding a specified level with durations greater than a specified duration.
 
This is not an exhaustive list of DURANL functions, but provides the flavor of its capability.
 
Perform this analysis (post run) using Pandas with these general opeations:

 + Read timeseries from HDF5 file using Pandas.
 + Aggregate/Disaggregate to the desired time steps (generally daily, monthly, annually) possibly using anchored values (such as annual starting in June),
 + Truncate to desired time period if necessary,
 + Use Pandas to create boolean arrays answering the question of levels,
 + Use a function such as runlength (shown below) to count the duration of an excursion,
 + Use Pandas tools such as histograms, selecting values above a certain duration, statistics.

The following example uses a temperature timeseries in test10.h5
 

##### Example function to calculate run lengths, returns sorted count list of run lengths as Pandas series.

In [ ]:
def runlength(boolean):
    durations = []
    counter   = 0
    
    for b in boolean:
        if b:
            counter += 1
        elif counter > 0:
            durations.append(counter)
            counter = 0
    
    # catch run at end that doesn't return to zero
    if counter > 0:
        durations.append(counter)
        
    return pd.Series(sorted(durations))

##### Read example data

In [ ]:
hdfname = os.path.join('TutorialData', 'Tutorial.h5')

In [ ]:
ds = pd.read_hdf(hdfname, 'TIMESERIES/TS122')
ds.head()

##### Create boolean expersion for exceeding levels

In [ ]:
boolean = ds > 32.0
boolean.head()

**Note:**  more complex expressions can represent excursion between two levels.

In [ ]:
b =  (32.0 < ds) & (ds <= 50.0)

b.head()

### Now answer DURANL like questions

Percentage of time above this level (32.0)

In [ ]:
print 100.0 * sum(boolean)/len(boolean)

Percentage of time between 32.0 and 50.0 (inclusive). (Note: this uses the b array computed above)

In [ ]:
print 100.0 * sum(b)/len(b)

Calculate all the runs (count consecutive boolean True values). Returns sorted Pandas series.

In [ ]:
runs = runlength(boolean)

print runs.head()
print
print runs.tail()

Show histogram of run lengths (for excursions > 32.0)

In [ ]:
plt.hist(runs)

Exclude last big run value (summer)

In [ ]:
plt.hist(runs[:-1])

How many runs greater than 50 in duration?

In [ ]:
print sum(runs >  50)

Number of runs with duration between 50 and 100 remembering these represent excursions above 32.0:

In [ ]:
print sum( (runs > 50) & (runs < 100))

Statistics on durations for excusion above 32.0

In [ ]:
runs[(runs > 50) & (runs < 100)].describe()

**Section Summary**

  + demonstrated that the HDF5 files can be easily processed (post run) to answer DURANL like questions.

## Section 4: HSP2 PLTGEN Functionality<a id='section4'></a>

**PLTGEN File Format Assumptions**

 + Text, not binary file
 + Initial 4 characters can be ignored in each line
 + First 25 lines are header information
 
To find the column header information

 + The line containing the word "LINTYP" immediately proceeds the column header lines
 + The column headers stop at the first of
     + line 26
     + blank line (ignoring the first 4 characters)
     + finding a line starting with "Time series" (ignoring the first 4 characters)

To find the data (columns of time series data)

 + Line 26 is dummy data
 + Line 27 and on are actual lines with time series data
 + No entry is blank => all lines have the same number of entries (columns)

     

##### Read a PLTGEN file

In [ ]:
pltgen = os.path.join('TutorialData', 'rch900.6')
df = HSP2tools.readPLTGEN(pltgen)
df

While the units are great, they make a complicated column header for working with the
data and the column names do not follow the Natural Naming convention.

In [ ]:
df['FLOW (ft3/s)'].plot(figsize=[20,10])

Accessing a column from the DataFrame (like above), produces a time series
(Pandas Series). This allows resampling to other periods such as monthly and
annually (as shown in Tutorial 3). The Pandas resampling methods include mean, sum, last, first, max, and min which cover the PLTGEN methods.  (PLTGEN AVER is Pandas mean.)  Pandas actually provides many more methods and allows user defined methods.

Calculate the annual flow:

In [ ]:
ts =  df['FLOW (ft3/s)']
ts = ts.resample('A').sum()
ts

Pandas can work with the entire DataFrame without needing to extract individual time series. Most functions also work with NAN's for missing data (like MATLAB.)

In [ ]:
df.describe()

Note the count above reflects the real values by ignoring NaNs.

**Section Summary**

 + demonstrated that HSP2 can read legacy PLTGEN files into Pandas DataFrames for analysis, plotting, and reporting